# Calcium Oscillation Modelling

## Model description

#### Model Overview

This article introduces a two-chamber model for explaining the dynamics of calcium ions (Ca²⁺) in frog skeletal muscle fibers.

#### Description

- **Two-chamber model:** The model consists of two compartments, the sarcoplasmic reticulum (SR) and the cytoplasm, with the muscle fibers located in the cytoplasm.
- **Ca²⁺ binding proteins:** The model simulates various Ca²⁺ binding proteins in the cytoplasm, including troponin, calsequestrin, and potentially added exogenous buffers (such as EGTA or BAPTA). The primary calcium binding protein considered in the SR is calsequestrin (CSQ), with its characteristics modeled based on experimental data.
- **R calcium buffering:** The buffering effect of Ca²⁺ in the SR exhibits cooperativity, following the Hill equation. Parameters such as the total number of Ca²⁺ binding sites and the half-saturation concentration change with the initial Ca²⁺ concentration in the SR.
- **Calcium release mechanism:** The calcium release from the SR is activated by a voltage-dependent mechanism, achieved by increasing the Ca²⁺ permeability of the SR membrane.

#### Key Features

- **Voltage-activated Ca²⁺ permeability:** The model uses experimentally obtained permeability time courses and amplitudes to drive the simulation.
- **Ca²⁺ buffering system:** The characteristics of the Ca²⁺ buffering system in the SR change under low initial [Ca²⁺]SR conditions, exhibiting a higher buffering capacity.
- **Experimental validation:** The model successfully reproduces the transient changes of Ca²⁺ in the SR and cytoplasm observed in experiments.
